In [ ]:
!pip install flask flask-ngrok sentence-transformers newspaper3k python-dotenv

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 65.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install beautifulsoup4

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_SEARCH_API_KEY")
print(GOOGLE_API_KEY)

69194c2d84d61f7bb4bc4e57c98ee761c48c8abd47e548551eba59f9c68e2b9f


In [ ]:
!pip install pyngrok

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok, conf
from sentence_transformers import SentenceTransformer, util
from newspaper import Article
import requests
import numpy as np
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import re

load_dotenv()

app = Flask(__name__)

conf.get_default().auth_token = os.getenv("NGROK_AUTH_TOKEN")

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print(f" * Ngrok tunnel URL: {public_url}")

# Load the sentence transformer model
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# SerpAPI (Google Search) key
SERPAPI_KEY = os.getenv("GOOGLE_SEARCH_API_KEY")

def search_google(query, count=3):
    params = {
        "engine": "google",
        "q": query,
        "num": count,
        "api_key": SERPAPI_KEY
    }
    try:
        response = requests.get("https://serpapi.com/search", params=params)
        results = response.json()
        urls = [item["link"] for item in results.get("organic_results", [])]
        return urls
    except Exception as e:
        print("Google search error:", e)
        return []

BLOCKED_DOMAINS = [
    "forbes.com", "sciencedirect.com", "reddit.com",
    "researchgate.net", "ieee.org", "springer.com",
    "facebook.com", "linkedin.com", "twitter.com",
    "instagram.com", "pinterest.com", "medium.com",
    "quora.com", "bloomberg.com", "wired.com",
    "nytimes.com", "washingtonpost.com"
]

def is_allowed(url):
    return not any(domain in url for domain in BLOCKED_DOMAINS)

def extract_text(url):
    if not is_allowed(url):
        print(f"❌ Skipping blocked domain: {url}")
        return ""
    try:
        print(f"🔍 Scraping: {url}")
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        for tag in soup(["script", "style", "footer", "nav", "noscript", "header", "meta"]):
            tag.decompose()

        text = soup.get_text(separator=" ")
        return text
    except Exception as e:
        print(f"⚠️ Error scraping {url}: {e}")
        return ""

def split_sentences(text):
    # Remove extra whitespace, HTML leftovers, etc.
    cleaned = re.sub(r'\s+', ' ', text).strip()

    # Basic sentence splitter (you can improve with NLP later)
    sentences = re.split(r'[.!?]', cleaned)

    return [s.strip() for s in sentences if s.strip()]

@app.route("/check/", methods=["GET"])
def check_route():
  return "Hello World"

@app.route("/check/", methods=["POST"])
def check_plagiarism():
    data = request.get_json()
    student_text = data.get("text", "")
    if not student_text:
        return jsonify({"error": "No text provided"}), 400

    student_sentences = split_sentences(student_text)
    student_embeddings = model.encode(student_sentences, convert_to_tensor=True)

    all_matches = []

    for sentence in student_sentences:
        urls = search_google(sentence)
        for url in urls:
            ref_text = extract_text(url)
            ref_sentences = split_sentences(ref_text)
            if not ref_sentences:
                continue

            ref_embeddings = model.encode(ref_sentences, convert_to_tensor=True)

            for i, s_emb in enumerate(student_embeddings):
                sim_scores = util.cos_sim(s_emb, ref_embeddings)[0]
                best_idx = int(np.argmax(sim_scores))
                best_score = float(sim_scores[best_idx])

                if best_score > 0.75:
                    all_matches.append({
                        "student_sentence": student_sentences[i],
                        "matched_sentence": ref_sentences[best_idx],
                        "similarity": round(best_score, 2),
                        "source_url": url
                    })

    return jsonify({"matches": all_matches})

app.run()

 * Ngrok tunnel URL: NgrokTunnel: "https://f3c240bbba8e.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Jul/2025 15:42:18] "GET /check/ HTTP/1.1" 200 -


🔍 Scraping: https://www.cnn.com/2025/07/12/us/timeline-texas-flooding


INFO:werkzeug:127.0.0.1 - - [15/Jul/2025 15:42:30] "POST /check/ HTTP/1.1" 200 -


❌ Skipping blocked domain: https://www.nytimes.com/live/2025/07/06/us/texas-floods
🔍 Scraping: https://www.latimes.com/california/story/2025-07-09/california-texas-flooding
🔍 Scraping: https://www.npr.org/2025/07/07/nx-s1-5459755/texas-floods-climate-change
🔍 Scraping: https://www.newsweek.com/texas-flash-floods-deaths-missing-storms-live-updates-2095388
🔍 Scraping: https://pmc.ncbi.nlm.nih.gov/articles/PMC9013542/
❌ Skipping blocked domain: https://www.sciencedirect.com/science/article/pii/S1470160X24004011
🔍 Scraping: https://6abc.com/post/search-texas-flooding-victims-paused-heavy-rain-prompts-kerr-county-flash-flood-warning-guadalupe-river-corridor-evacuated/17101757/
🔍 Scraping: https://www.nbcnews.com/news/us-news/texas-officials-defend-response-deadly-floods-saved-many-people-rcna218638
🔍 Scraping: https://gov.texas.gov/news/post/governor-abbott-continues-state-response-recovery-efforts-for-texans-impacted-by-flooding
🔍 Scraping: https://www.illinois.gov/news/press-release.30430

...

INFO:werkzeug:127.0.0.1 - - [15/Jul/2025 15:52:18] "POST /check/ HTTP/1.1" 200 -


❌ Skipping blocked domain: https://www.reddit.com/r/india/comments/1m0fyxf/as_theories_swirl_about_air_india_crash_key/
🔍 Scraping: https://www.bbc.com/news/articles/cp3lpdqp7x3o


INFO:werkzeug:127.0.0.1 - - [15/Jul/2025 15:57:58] "POST /check/ HTTP/1.1" 200 -


🔍 Scraping: https://www.espn.com/soccer/story/_/id/45732507/chelsea-win-fifa-club-world-cup-vs-psg-cole-palmer-mvp-reaction-highlights-analysis


INFO:werkzeug:127.0.0.1 - - [15/Jul/2025 15:59:57] "POST /check/ HTTP/1.1" 200 -
